# Phi-4 Mini ONNX Parallell Funktionsanrop Tutorial

Den här guiden visar hur man använder Phi-4 Mini med ONNX Runtime GenAI för parallella funktionsanrop. Funktionsanrop gör det möjligt för modellen att intelligent använda externa verktyg och API:er baserat på användarens förfrågningar.

## Översikt

I den här guiden kommer du att lära dig:
- Hur du ställer in Phi-4 Mini med ONNX Runtime GenAI
- Hur du definierar funktionsscheman för att boka flyg och hotell
- Hur du använder styrd generering med Lark-grammatik för strukturerad output
- Hur du utför parallella funktionsanrop för komplexa resebokningsscenarier

## Förutsättningar

Innan du kör den här guiden, se till att du:
- Har laddat ner Phi-4 Mini ONNX-modellen
- Har installerat paketet `onnxruntime-genai`
- Har grundläggande förståelse för konceptet funktionsanrop


## Steg 1: Importera nödvändiga bibliotek

Först importerar vi de bibliotek som behövs för att implementera vår funktionsanrop.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## Steg 2: Modellinställning och Konfiguration

Nu ska vi konfigurera Phi-4 Mini ONNX-modellen. Se till att ersätta sökvägen med din faktiska modellkatalog.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## Steg 3: Konfigurera genereringsparametrar

Ställ in genereringsparametrarna för att styra modellens beteende vid output. Dessa inställningar säkerställer deterministiska och fokuserade svar vid funktionsanrop.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## Steg 4: Definiera Tillgängliga Funktioner

Här definierar vi de funktioner som vår AI-assistent kan använda. I detta exempel har vi två rese-relaterade funktioner:
1. **booking_flight_tickets**: För att boka flyg mellan flygplatser
2. **booking_hotels**: För att boka hotellboende

Funktionerna definieras enligt OpenAI:s schema för funktionsanrop.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## Steg 5: Hjälpfunktioner för grammatikgenerering

Dessa hjälpfunktioner omvandlar våra funktionsdefinitioner till Lark-grammatikformat, vilket används av ONNX Runtime GenAI för styrd generering. Detta säkerställer att modellen genererar giltiga funktionsanrop i korrekt JSON-format.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## Steg 6: Testa grammatikgenerering

Låt oss testa våra funktioner för grammatikgenerering för att se hur de omvandlar våra verktygsdefinitioner till rätt format.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## Steg 7: Förbered systemprompt och generator

Nu ska vi skapa systemprompten som informerar modellen om tillgängliga verktyg och ställa in generatorn med styrda genereringsparametrar.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## Steg 8: Initiera Generator med Styrd Generering

Nu ska vi skapa generatorn med våra konfigurerade parametrar och tillämpa Lark-grammatiken för styrd generering.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## Steg 9: Testa parallell funktionsanrop

Nu ska vi testa vår setup med ett komplext resebokningsscenario som kräver anrop av flera funktioner.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### Generera Funktionsanrop

Modellen kommer nu att generera strukturerade funktionsanrop baserat på vår användarförfrågan. Tack vare styrd generering kommer output att vara i giltigt JSON-format som kan köras direkt.

**Förväntat Resultat**: Modellen bör generera funktionsanrop för:
1. `booking_flight_tickets` med detaljer för Beijing (PEK) till Paris (CDG)
2. `booking_hotels` med detaljer för boende i Paris

Kör cellen nedan för att se den live genereringen:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Slutsats

🎉 **Grattis!** Du har framgångsrikt implementerat parallell funktionsanrop med Phi-4 Mini med hjälp av ONNX Runtime GenAI.

### Vad du har lärt dig:

1. **Modellkonfiguration**: Hur man konfigurerar Phi-4 Mini med ONNX Runtime GenAI
2. **Funktionsdefinition**: Hur man definierar funktionsscheman för AI-funktionsanrop
3. **Styrd generering**: Hur man använder Lark-grammatik för strukturerad utdata
4. **Parallella funktionsanrop**: Hur man hanterar komplexa förfrågningar som kräver flera funktionsanrop

### Viktiga fördelar:

- ✅ **Strukturerad utdata**: Styrd generering säkerställer giltiga JSON-funktionsanrop
- ✅ **Parallell bearbetning**: Hantera flera funktionsanrop i en enda förfrågan
- ✅ **Hög prestanda**: ONNX Runtime erbjuder optimerad inferens
- ✅ **Flexibelt schema**: Lätt att lägga till eller ändra funktionsdefinitioner

### Resurser:

- [Phi-4 Mini Dokumentation](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI Dokumentation](https://onnxruntime.ai/docs/genai/)
- [Bästa praxis för funktionsanrop](https://platform.openai.com/docs/guides/function-calling)



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör det noteras att automatiserade översättningar kan innehålla fel eller brister. Det ursprungliga dokumentet på dess originalspråk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
